[View in Colaboratory](https://colab.research.google.com/github/uranus666/keras_cnn_cat_dog/blob/master/keras_cnn_cat_dog.ipynb)

In [0]:
!pip install -U -q PyDrive


In [0]:
# if you want to use data from drive for colab

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import os

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
#To get id,  Enable link sharing for the file you want to transfer. Copy the link
download = drive.CreateFile({'id': '<paste file id here>'})
download.GetContentFile('< name of zip file >')

In [0]:
# unzip file to use
!unzip <name of zip file>.zip

In [0]:
# import Libraries
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Sequential,load_model
from keras.layers import Dense,Conv2D, MaxPooling2D,Flatten, Activation, Dropout
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras import optimizers
import numpy as np
import matplotlib.pyplot as plt

In [0]:
# Getting Dataset (kaggle dog and cat used 10000 sample, 8000 train 2000 test)
train_dir = '/train_data'
test_dir = '/test_data'

In [21]:
# data augmentation

train_data = ImageDataGenerator(rescale=1./255,
                                rotation_range=40,
                                width_shift_range=0.2,
                                height_shift_range=0.2,
                                shear_range=0.2,
                                zoom_range=0.2,
                                horizontal_flip=True)

test_data = ImageDataGenerator(rescale=1./255)


train_generator = train_data.flow_from_directory(train_dir,
                                             target_size=(150,150),
                                             batch_size=20,
                                             class_mode='binary')
test_generator = test_data.flow_from_directory(test_dir,
                                         target_size=(150,150),
                                         batch_size=20,
                                         class_mode='binary')

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [0]:
callbacks = [EarlyStopping(monitor='val_loss', patience=5),
             ModelCheckpoint(filepath='best_model_cnn.cat_dogs.h5', monitor='val_loss', save_best_only=True)]

In [0]:
# Creating Model
model = Sequential()
model.add(Conv2D(32, (3, 3),input_shape=(150, 150, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('sigmoid'))


model.compile(optimizer=optimizers.RMSprop(lr=1e-4), loss='binary_crossentropy',metrics=['acc'])


In [0]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=100,
    callbacks=callbacks,
    validation_data=test_generator,
    validation_steps=50
)

In [0]:
n_model = load_model('best_model_cnn.cat_dogs.h5') # load model to reuse

In [0]:
# prepare new img for predict
def prepare_img(img_path):
    img = image.load_img(img_path, target_size=(150,150))
    img_tensor = image.img_to_array(img)
    img_tensor = np.expand_dims(img_tensor, axis=0)
    img_tensor /= 255.
    return img_tensor

new_img = prepare_img(img_path)

plt.imshow(new_img[0])
plt.show()